In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore',category = DeprecationWarning)

In [ ]:
df = pd.read_csv('/content/order_item_feedback_original.csv')


In [ ]:
df.head()

In [ ]:
# Concatenate the 'review_title' and 'review_details' columns
df['combined_review'] = df['review_title'] + ' ' + df['review_details']

In [ ]:
review_df= df[['combined_review']]

In [ ]:
review_df['combined_review'].isna().sum()

1

In [ ]:
review_without_nan = review_df.copy()
review_without_nan.dropna(inplace=True)

## **Text preprocessing**

###Lower casing

In [ ]:
#lower casing the text
review_without_nan['combined_review'] = review_without_nan['combined_review'].str.lower()

### Remove html tags

In [ ]:
#funtion to remove html tags gerenerally we use
import re
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return re.sub(pattern, '', text)

In [ ]:
#removing the html tags
review_without_nan['combined_review'] = review_without_nan['combined_review'].apply(remove_html_tags)

###URL Handling

In [ ]:
def remove_urls(text):
  pattern= re.compile(r'https?://\S+|www\.\S+')
  return pattern.sub(r'', text)

In [ ]:
review_without_nan['combined_review'] = review_without_nan['combined_review'].apply(remove_urls)

### Punctuation Removal

In [ ]:
import string
import re
import time
import pandas as pd

def improved_text_cleaning(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove 'n' when it's not part of a word
    text = re.sub(r'\s+n\s+', ' ', text)

    # Remove extra spaces
    text = ' '.join(text.split())

    return text

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
#Punctuation removal apply
review_without_nan['combined_review'] = review_without_nan['combined_review'].apply(improved_text_cleaning)

### Chat word removal

In [ ]:
chat_words_dict = {
    "brb": "Be Right Back",
    "lol": "Laugh Out Loud",
    "rofl": "Rolling On the Floor Laughing",
    "omg": "Oh My God",
    "ttyl": "Talk To You Later",
    "idk": "I Don’t Know",
    "smh": "Shaking My Head",
    "imo": "In My Opinion",
    "imho": "In My Humble Opinion",
    "fyi": "For Your Information",
    "btw": "By The Way",
    "nvm": "Never Mind",
    "tbh": "To Be Honest",
    "gtg": "Got To Go",
    "bff": "Best Friends Forever",
    "jk": "Just Kidding",
    "ttfn": "Ta-Ta For Now",
    "yolo": "You Only Live Once",
    "afk": "Away From Keyboard",
    "irl": "In Real Life",
    "dm": "Direct Message",
    "pm": "Private Message",
    "gg": "Good Game",
    "ftw": "For The Win",
    "ppl": "People",
    "thx": "Thanks",
    "lmao": "Laughing My Ass Off",
    "icymi": "In Case You Missed It",
    "fomo": "Fear Of Missing Out",
    "omw": "On My Way",
    "idc": "I Don’t Care",
    "asap": "As Soon As Possible",
    "b/c": "Because",
    "cul8r": "See You Later",
    "gr8": "Great",
    "k": "Okay",
    "l8r": "Later",
    "np": "No Problem",
    "tmi": "Too Much Information",
    "ty": "Thank You",
    "yw": "You’re Welcome",
    "wtf": "What The Fuck",
    'u': 'you',
    'ur': 'your',
    'r': 'are',
    'lol': 'laughing out loud',
    'omg': 'oh my god',
    'idk': 'I do not know',
    'btw': 'by the way',
    'brb': 'be right back',
    'b4': 'before',
    'gr8': 'great',
    'bday': 'birthday',
    'plz': 'please',
    'thx': 'thanks',
    'np': 'no problem',
    'cuz': 'because',
    'msg': 'message',
    'wanna': 'want to',
    'gonna': 'going to'
}


In [ ]:
import re

def replace_chat_words(text, chat_words_dict):
    chat_words_re = re.compile(r'\b(' + '|'.join(chat_words_dict.keys()) + r')\b')
    return chat_words_re.sub(lambda x: chat_words_dict[x.group()], text)

In [ ]:
# Define the function to preprocess the text
def preprocess_text(text):
    # Replace chat words
    text = replace_chat_words(text, chat_words_dict)
    return text

In [ ]:
review_without_nan['combined_review'] = review_without_nan['combined_review'].apply(preprocess_text)

### Emoji handler

In [ ]:
!pip install emoji
import emoji
# Function to replace emojis with text descriptions
def replace_emojis(text):
    return emoji.demojize(text)
review_without_nan['combined_review'] = review_without_nan['combined_review'].apply(replace_emojis)

###Stop word Removal

In [ ]:
pip install nltk langdetect indic-nlp-library

In [ ]:
import re
from langdetect import detect
from nltk.corpus import stopwords
from indicnlp.tokenize import indic_tokenize

# English stopwords (from NLTK)
english_stopwords = set(stopwords.words('english'))

# Hinglish stopwords (custom list)
hinglish_stopwords = set(['hai', 'hain', 'ka', 'ki', 'ke', 'kar', 'mein', 'tha', 'raha', 'thi', 'hota', 'ki'])

# Hindi stopwords (custom list or from a Hindi stopword corpus)
hindi_stopwords = set(['है', 'और', 'का', 'में', 'से', 'को', 'कि', 'तो', 'यह', 'पर', 'था', 'हूँ'])

# Function to detect language and remove corresponding stopwords
def remove_stopwords_multilingual(review):
    try:
        # Detect the language of the review
        lang = detect(review)

        # Tokenize the review into words
        words = review.split()

        if lang == 'en':  # English
            return ' '.join([word for word in words if word.lower() not in english_stopwords])
        elif lang == 'hi':  # Hindi
            # Tokenize Hindi text with indicnlp
            words = indic_tokenize.trivial_tokenize(review)
            return ' '.join([word for word in words if word not in hindi_stopwords])
        else:  # Assume Hinglish for unknown language or if Hinglish detected
            return ' '.join([word for word in words if word.lower() not in hinglish_stopwords])

    except Exception as e:
        print(f"Error processing review: {review}, Error: {str(e)}")
        return review  # Return original text if error occurs

In [ ]:
# # Function to remove stopwords from text without using word_tokenize
# def remove_stopwords_multilingual(text):
#     # Split the text into words based on spaces
#     words = text.split()
#     # Filter out stopwords
#     filtered_words = [word for word in words if word.lower() not in stop_words]
#     # Join the filtered words back into a single string
#     filtered_text = ' '.join(filtered_words)
#     return filtered_text

In [ ]:
#stopword removal
review_without_nan['combined_review'] = review_without_nan['combined_review'].apply(remove_stopwords_multilingual)

### Final processing

In [ ]:
from nltk import word_tokenize
import nltk
nltk.download('punkt')
review_without_nan['combined_review'] = review_without_nan['combined_review'].apply(lambda x:' '.join([word for word in word_tokenize(x) if word not in stop_words and len(word)> 2]))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
review_without_nan

### Spell checker

In [ ]:
!pip install pyspellchecker

In [ ]:
import pandas as pd
from spellchecker import SpellChecker

# Initialize the spell checker
spell = SpellChecker()

# Function to correct spelling in a sentence
def correct_spelling(text):
    if pd.isna(text) or not isinstance(text, str):
        return text  # Return the original value if it's NaN or not a string

    # Tokenize the text into words
    words = text.split()
    # Find misspelled words
    misspelled_words = spell.unknown(words)
    # Correct the misspelled words and ensure all are strings
    corrected_words = [spell.correction(word) if word in misspelled_words else word for word in words]
    # Filter out None values and ensure all are strings
    corrected_words = [word if isinstance(word, str) else '' for word in corrected_words]
    # Join corrected words into a sentence
    return ' '.join(corrected_words)

# Apply the correction function to the 'combined_review' column
review_without_nan['combined_review_corrected'] = review_without_nan['combined_review'].apply(correct_spelling)

# Show the updated DataFrame
review_without_nan[['combined_review', 'combined_review_corrected']].head()

,combined_review,combined_review_corrected
0,effective never thought something could work u...,effective never thought something could work u...
1,love malai real sense blends skin make skin su...,love malar real sense blends skin make skin su...
2,amazing packs wow using almost 15days love nut...,amazing packs wow using almost days love nut h...
3,heavenly smell smells amazing moisturising gre...,heavenly smell smells amazing moisturizing gre...
4,spa like feel lovely face pack quite versatile...,spa like feel lovely face pack quite versatile...


In [ ]:
# !pip install textblob
# from textblob import TextBlob
# def correct_spelling(text):
#     return str(TextBlob(text).correct())
# # Apply the spell checker to the 'review_details' column
# review_without_nan['combined_review'] = review_without_nan['combined_review'].apply(correct_spelling)

### VADER SENTIMENT ANALYSIS

In [ ]:
review_without_nan

### Final preprocessing

In [ ]:
# Export to a CSV file
review_without_nan.to_csv('final_df_export.csv', index=False)

In [ ]:
!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
# Define a function to apply VADER to each review
def get_vader_sentiment(text):
    return analyzer.polarity_scores(text)

In [ ]:
# Apply the function to your review details column
review_without_nan['vader_sentiment'] = review_without_nan['combined_review_corrected'].apply(get_vader_sentiment)

In [ ]:
review_without_nan

In [ ]:
# Extract individual sentiment scores into separate columns
review_without_nan['vader_neg'] = review_without_nan['vader_sentiment'].apply(lambda x: x['neg'])
review_without_nan['vader_neu'] = review_without_nan['vader_sentiment'].apply(lambda x: x['neu'])
review_without_nan['vader_pos'] = review_without_nan['vader_sentiment'].apply(lambda x: x['pos'])
review_without_nan['vader_compound'] = review_without_nan['vader_sentiment'].apply(lambda x: x['compound'])


In [ ]:
# Drop the 'vader_scores' column if you no longer need the dictionary
review_without_nan.drop(columns=['vader_sentiment'], inplace=True)

In [ ]:
review_without_nan.head()

,combined_review,combined_review_corrected,vader_neg,vader_neu,vader_pos,vader_compound
0,effective never thought something could work u...,effective never thought something could work u...,0.0,0.638,0.362,0.7783
1,love malai real sense blends skin make skin su...,love malar real sense blends skin make skin su...,0.0,0.576,0.424,0.8442
2,amazing packs wow using almost 15days love nut...,amazing packs wow using almost days love nut h...,0.0,0.306,0.694,0.9591
3,heavenly smell smells amazing moisturising gre...,heavenly smell smells amazing moisturizing gre...,0.0,0.202,0.798,0.9501
4,spa like feel lovely face pack quite versatile...,spa like feel lovely face pack quite versatile...,0.0,0.447,0.553,0.9741


In [ ]:
review_without_nan['vader_compound'][190]

0.7906

In [ ]:
# Define a function to classify sentiment based on the compound score
def classify_sentiment(compound):
    if compound > 0.05:
        return 'Positive'
    elif compound < -0.05:
        return 'Negative'
    else:
        return 'Neutral'

In [ ]:
# Apply the function to the 'vader_compound' column to create a new 'sentiment' column
review_without_nan['sentiment'] = review_without_nan['vader_compound'].apply(classify_sentiment)

In [ ]:
review_without_nan['sentiment'].value_counts()

,count
sentiment,
Positive,869
Negative,121
Neutral,69


In [ ]:
review_without_nan

###ROBERTA APPLICATION

In [ ]:
roberta = pd.read_csv('/content/final_df_export.csv')

In [ ]:
roberta

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

sia = SentimentIntensityAnalyzer()

In [ ]:
from tqdm import tqdm
import pandas as pd

# Assuming 'polarity_scores_roberta' is already defined
# Define a dictionary to hold the results
res = {}

# Iterate through the DataFrame rows
for i, row in tqdm(roberta.iterrows(), total=len(roberta)):
    try:
        # Extract the necessary fields
        text = row['combined_review_corrected']
        myid = i  # Use the row index as the ID

        # Check if the text is a string
        if isinstance(text, str):
            # Get RoBERTa sentiment scores
            roberta_result = polarity_scores_roberta(text)

            # Combine the RoBERTa results
            res[myid] = roberta_result
        else:
            print(f'Skipped non-string input at index {myid}')
    except RuntimeError:
        print(f'Broke for index {myid}')

# Convert the results dictionary back to a DataFrame
results_df = pd.DataFrame.from_dict(res, orient='index')

# Merge the results back with the original DataFrame
final_df = pd.concat([roberta, results_df], axis=1)


 58%|█████▊    | 611/1059 [01:42<00:53,  8.43it/s]

Skipped non-string input at index 609


 72%|███████▏  | 758/1059 [02:01<00:25, 11.68it/s]

Skipped non-string input at index 755


 72%|███████▏  | 767/1059 [02:02<00:26, 10.90it/s]

Skipped non-string input at index 765


100%|██████████| 1059/1059 [02:39<00:00,  6.66it/s]


In [ ]:
final_df.head()

,combined_review,combined_review_corrected,roberta_neg,roberta_neu,roberta_pos
0,effective never thought something could work u...,effective never thought something could work u...,0.026363,0.179872,0.793765
1,love malai real sense blends skin make skin su...,love malar real sense blends skin make skin su...,0.004482,0.085154,0.910364
2,amazing packs wow using almost 15days love nut...,amazing packs wow using almost days love nut h...,0.002192,0.013398,0.984410
3,heavenly smell smells amazing moisturising gre...,heavenly smell smells amazing moisturizing gre...,0.001917,0.011140,0.986943
4,spa like feel lovely face pack quite versatile...,spa like feel lovely face pack quite versatile...,0.002776,0.013783,0.983442


In [ ]:
# Example to calculate a compound score based on RoBERTa outputs
def calculate_roberta_compound(roberta_neg, roberta_neu, roberta_pos):
    return roberta_pos - roberta_neg

# Assuming you have the DataFrame with RoBERTa results:
final_df['roberta_compound'] = final_df.apply(
    lambda row: calculate_roberta_compound(row['roberta_neg'], row['roberta_neu'], row['roberta_pos']),
    axis=1
)

In [ ]:
final_df

,combined_review,combined_review_corrected,roberta_neg,roberta_neu,roberta_pos,roberta_compound
0,effective never thought something could work u...,effective never thought something could work u...,0.026363,0.179872,0.793765,0.767402
1,love malai real sense blends skin make skin su...,love malar real sense blends skin make skin su...,0.004482,0.085154,0.910364,0.905882
2,amazing packs wow using almost 15days love nut...,amazing packs wow using almost days love nut h...,0.002192,0.013398,0.984410,0.982217
3,heavenly smell smells amazing moisturising gre...,heavenly smell smells amazing moisturizing gre...,0.001917,0.011140,0.986943,0.985025
4,spa like feel lovely face pack quite versatile...,spa like feel lovely face pack quite versatile...,0.002776,0.013783,0.983442,0.980666
...,...,...,...,...,...,...
1054,less quantity received less quanity bottle com...,less quantity received less quality bottle com...,0.508129,0.443443,0.048428,-0.459700
1055,awesome love lepa,awesome love leap,0.004878,0.048388,0.946734,0.941856
1056,best face wash love itpure natural product,best face wash love impure natural product,0.004080,0.053943,0.941977,0.937897
1057,worst experience delivery late packaging open ...,worst experience delivery late packaging open ...,0.941349,0.054239,0.004412,-0.936937


In [ ]:
# Apply the function to the 'vader_compound' column to create a new 'sentiment' column
final_df['sentiment_roberta'] = final_df['roberta_compound'].apply(classify_sentiment)

In [ ]:
final_df

### Comparison

In [ ]:

final_df.reset_index(drop=True, inplace=True)
review_without_nan.reset_index(drop=True, inplace=True)

# Concatenate columns from 'review_without_nan' into 'final_df'
final_df = pd.concat([
    final_df[['combined_review_corrected', 'roberta_compound', 'sentiment_roberta']],
    review_without_nan[['vader_compound', 'sentiment']]
], axis=1)


In [ ]:
# Show the updated DataFrame
final_df.head()

In [ ]:
final_df.sample(20)

,combined_review_corrected,roberta_compound,sentiment_roberta,vader_compound,sentiment
323,appreciation lovely gift thank much not habit ...,0.985140,Positive,0.9802,Positive
625,less quantity packet small expected,-0.110531,Negative,0.0000,Neutral
543,nice hair color better hair texture post usage...,0.971705,Positive,0.9893,Positive
364,love product seen difference first,0.655720,Positive,0.6369,Positive
244,refreshing gulag jail bought jail month ago us...,0.151394,Positive,0.0000,Neutral
623,feel good love gel skin feel plum moisturizes,0.961195,Positive,0.7964,Positive
504,good product sticky dulls complexion,-0.181652,Negative,0.2263,Positive
574,pure aloe very gel ordered items together aloe...,0.795944,Positive,0.9773,Positive
892,puffed package product came puff package smell...,0.963233,Positive,0.4404,Positive
945,product liked texture smell pack didn't notice...,0.152572,Positive,0.4215,Positive


In [ ]:
final_df['combined_review_corrected'][244]

'refreshing gulag jail bought jail month ago using regularly night keeps oil control leaves skin refreshing'

In [ ]:
# Extract rows where sentiment_roberta and sentiment are not equal
mismatched_sentiments = final_df[final_df['sentiment_roberta'] != final_df['sentiment']]

In [ ]:
mismatched_sentiments.shape

(140, 5)

In [ ]:
mismatched_sentiments

,combined_review_corrected,roberta_compound,sentiment_roberta,vader_compound,sentiment
5,okay okay sharp loofah almost like dry brush b...,-0.683011,Negative,0.3415,Positive
12,one pack missing ordered pack nutrias 250 big ...,0.016911,Neutral,-0.2960,Negative
13,okay okay product nourish lips don't give natu...,-0.386703,Negative,0.1754,Positive
19,misleading product said enhance hair growth us...,-0.385622,Negative,0.3182,Positive
23,works thank not habit amazing products underar...,0.827924,Positive,-0.1461,Negative
...,...,...,...,...,...
1030,killer face gel first got know power product d...,0.862936,Positive,-0.5106,Negative
1035,baby comb darn quality,-0.638654,Negative,0.0000,Neutral
1049,quality unto mark ordering product regularly s...,-0.816915,Negative,0.0000,Neutral
1050,,-0.092669,Negative,0.0000,Neutral


In [ ]:
mismatched_sentiments['combined_review_corrected'][1054]

'less quantity received less quality bottle completely full'